In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from scipy.ndimage import rotate
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
def flip_image(image):
    return tf.image.flip_left_right(image).numpy()

In [3]:
def rotate_image(image,angle):
    return rotate(image,angle)

In [4]:
def generate_tabular_dataset(srcpath,name):

    exercices = os.listdir(srcpath)
    rotations = [-5,-3,-1,0,1,3,5]


    if not os.path.exists(srcpath + '/' + name + '.csv') :
        data_train = pd.DataFrame(columns = ['NOSE_X', 
                                             'NOSE_Y',
                                             'NOSE_Z',
                                             'LEFT_SHOULDER_X',
                                             'LEFT_SHOULDER_Y',
                                             'LEFT_SHOULDER_Z',
                                             'RIGHT_SHOULDER_X',
                                             'RIGHT_SHOULDER_Y',
                                             'RIGHT_SHOULDER_Z',
                                             'LEFT_ELBOW_X',
                                             'LEFT_ELBOW_Y',
                                             'LEFT_ELBOW_Z',
                                             'RIGHT_ELBOW_X',
                                             'RIGHT_ELBOW_Y',
                                             'RIGHT_ELBOW_Z',
                                             'LEFT_WRIST_X',
                                             'LEFT_WRIST_Y',
                                             'LEFT_WRIST_Z',
                                             'RIGHT_WRIST_X',
                                             'RIGHT_WRIST_Y',
                                             'RIGHT_WRIST_Z',
                                             'LEFT_PINKY_X',
                                             'LEFT_PINKY_Y',
                                             'LEFT_PINKY_Z',
                                             'RIGHT_PINKY_X',
                                             'RIGHT_PINKY_Y',
                                             'RIGHT_PINKY_Z',
                                             'LEFT_INDEX_X',
                                             'LEFT_INDEX_Y',
                                             'LEFT_INDEX_Z',
                                             'RIGHT_INDEX_X',
                                             'RIGHT_INDEX_Y',
                                             'RIGHT_INDEX_Z',
                                             'LEFT_THUMB_X',
                                             'LEFT_THUMB_Y',
                                             'LEFT_THUMB_Z',
                                             'RIGHT_THUMB_X',
                                             'RIGHT_THUMB_Y',
                                             'RIGHT_THUMB_Z',
                                             'LEFT_HIP_X',
                                             'LEFT_HIP_Y',
                                             'LEFT_HIP_Z',
                                             'RIGHT_HIP_X',
                                             'RIGHT_HIP_Y',
                                             'RIGHT_HIP_Z',
                                             'LEFT_KNEE_X',
                                             'LEFT_KNEE_Y',
                                             'LEFT_KNEE_Z',
                                             'RIGHT_KNEE_X',
                                             'RIGHT_KNEE_Y',
                                             'RIGHT_KNEE_Z',
                                             'LEFT_ANKLE_X',
                                             'LEFT_ANKLE_Y',
                                             'LEFT_ANKLE_Z',
                                             'RIGHT_ANKLE_X',
                                             'RIGHT_ANKLE_Y',
                                             'RIGHT_ANKLE_Z',
                                             'LEFT_HEEL_X', 
                                             'LEFT_HEEL_Y',
                                             'LEFT_HEEL_Z',
                                             'RIGHT_HEEL_X', 
                                             'RIGHT_HEEL_Y',
                                             'RIGHT_HEEL_Z',
                                             'LEFT_FOOT_INDEX_X', 
                                             'LEFT_FOOT_INDEX_Y',
                                             'LEFT_FOOT_INDEX_Z',
                                             'RIGHT_FOOT_INDEX_X', 
                                             'RIGHT_FOOT_INDEX_Y',
                                             'RIGHT_FOOT_INDEX_Z',
                                             'ID',
                                             'EXERCICE'])

    else : 
        data_train = pd.read_csv(srcpath + '/' + name + '.csv')

    
    for ex in exercices : 
        err_count = 0
        count = 0
        if 'csv' not in ex :
            images = os.listdir(srcpath + '/' + ex)
            print('Processing exercice ' + ex)
            if len(images):
                with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

                    for im in tqdm(images) :
                        path = srcpath + '/' + ex + '/' + im
                        # image augmentation 
                        for transf in rotations :
                            ID_image = path + '_' + str(transf)
                            count += 1
                            if ID_image not in data_train.ID.unique() :
                                try :
                                
                                    image = cv2.imread(path)
                                    image.flags.writeable = False
                                    image = rotate_image(image,transf)


                                    # flip : 

                                    for i in range(2):
                                        if i :
                                            image = flip_image(image)
                                            ID_image = ID_image + '_fliped'


                                        results = pose.process(image)
                                        landmarks = results.pose_landmarks.landmark

                                        new_entry = {'NOSE_X' : landmarks[mp_pose.PoseLandmark.NOSE.value].x, 
                                                    'NOSE_Y' : landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                                    'NOSE_Z' : landmarks[mp_pose.PoseLandmark.NOSE.value].z,
                                                    'LEFT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                                    'LEFT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                                    'LEFT_SHOULDER_Z' : landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z,
                                                    'RIGHT_SHOULDER_X' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                                    'RIGHT_SHOULDER_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                                    'RIGHT_SHOULDER_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z,
                                                    'LEFT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                                    'LEFT_ELBOW_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                                                    'LEFT_ELBOW_Z' : landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z,
                                                    'RIGHT_ELBOW_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                                    'RIGHT_ELBOW_Y': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                                                    'RIGHT_ELBOW_Z': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z,
                                                    'LEFT_WRIST_X' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                                    'LEFT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                                                    'LEFT_WRIST_Z' : landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z,
                                                    'RIGHT_WRIST_X' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                                    'RIGHT_WRIST_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,
                                                    'RIGHT_WRIST_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z,
                                                    'LEFT_PINKY_X' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,
                                                    'LEFT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y,
                                                    'LEFT_PINKY_Z' : landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].z,
                                                    'RIGHT_PINKY_X' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,
                                                    'RIGHT_PINKY_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y,
                                                    'RIGHT_PINKY_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].z,
                                                    'LEFT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                                                    'LEFT_INDEX_Y': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y,
                                                    'LEFT_INDEX_Z': landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].z,
                                                    'RIGHT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,
                                                    'RIGHT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y,
                                                    'RIGHT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].z,
                                                    'LEFT_THUMB_X' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,
                                                    'LEFT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y,
                                                    'LEFT_THUMB_Z' : landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].z,
                                                    'RIGHT_THUMB_X' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,
                                                    'RIGHT_THUMB_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y,
                                                    'RIGHT_THUMB_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].z,
                                                    'LEFT_HIP_X' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                                    'LEFT_HIP_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                                                    'LEFT_HIP_Z' : landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z,
                                                    'RIGHT_HIP_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                                    'RIGHT_HIP_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                                    'RIGHT_HIP_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z,
                                                    'LEFT_KNEE_X' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                                    'LEFT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                                                    'LEFT_KNEE_Z' : landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z,
                                                    'RIGHT_KNEE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                                    'RIGHT_KNEE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                                    'RIGHT_KNEE_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z,
                                                    'LEFT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                                    'LEFT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                                                    'LEFT_ANKLE_Z' : landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z,
                                                    'RIGHT_ANKLE_X' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                                    'RIGHT_ANKLE_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                                    'RIGHT_ANKLE_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z,
                                                    'LEFT_HEEL_X' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,
                                                    'LEFT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y,
                                                    'LEFT_HEEL_Z' : landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z,
                                                    'RIGHT_HEEL_X' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, 
                                                    'RIGHT_HEEL_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y,
                                                    'RIGHT_HEEL_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z,
                                                    'LEFT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,  
                                                    'LEFT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y,  
                                                    'LEFT_FOOT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z,  
                                                    'RIGHT_FOOT_INDEX_X' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                                    'RIGHT_FOOT_INDEX_Y' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,
                                                    'RIGHT_FOOT_INDEX_Z' : landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,
                                                    'ID' : ID_image,
                                                    'EXERCICE' : ex}

                                        data_train = data_train.append(new_entry,ignore_index=True)
                                except :
                                    err_count+=1

                print('nb error (missed opport) / total if no error : {}/{} '.format(err_count,count))
                data_train.to_csv(srcpath + '/' + name + '.csv',index=False)

    return data_train




In [5]:
df_train = generate_tabular_dataset('./Data_Train','train_z_augm')

Processing exercice dips


100%|██████████| 302/302 [08:10<00:00,  1.63s/it]


nb error (missed opport) / total if no error : 660/2114 
Processing exercice pushup


100%|██████████| 683/683 [06:49<00:00,  1.67it/s]


nb error (missed opport) / total if no error : 2279/4781 
Processing exercice situp


 37%|███▋      | 351/959 [03:34<10:53,  1.07s/it]

In [ ]:
df_test = generate_tabular_dataset('./Data_Test','test_z_augm')

Processing exercice dips


100%|██████████| 53/53 [00:17<00:00,  3.08it/s]


Processing exercice pushup


100%|██████████| 303/303 [00:18<00:00, 16.56it/s]


Processing exercice situp


100%|██████████| 521/521 [00:32<00:00, 16.09it/s]


Processing exercice squats


 34%|███▍      | 25/73 [00:10<00:30,  1.60it/s]

In [ ]:
X_cols = ['NOISE_X',
 'NOISE_Y',
 'NOISE_Z',
 'LEFT_SHOULDER_X',
 'LEFT_SHOULDER_Y',
 'LEFT_SHOULDER_Z',
 'RIGHT_SHOULDER_X',
 'RIGHT_SHOULDER_Y',
 'RIGHT_SHOULDER_Z',
 'LEFT_ELBOW_X',
 'LEFT_ELBOW_Y',
 'LEFT_ELBOW_Z',
 'RIGHT_ELBOW_X',
 'RIGHT_ELBOW_Y',
 'RIGHT_ELBOW_Z',
 'LEFT_WRIST_X',
 'LEFT_WRIST_Y',
 'LEFT_WRIST_Z',
 'RIGHT_WRIST_X',
 'RIGHT_WRIST_Y',
 'RIGHT_WRIST_Z',
 'LEFT_PINKY_X',
 'LEFT_PINKY_Y',
 'LEFT_PINKY_Z',
 'RIGHT_PINKY_X',
 'RIGHT_PINKY_Y',
 'RIGHT_PINKY_Z',
 'LEFT_INDEX_X',
 'LEFT_INDEX_Y',
 'LEFT_INDEX_Z',
 'RIGHT_INDEX_X',
 'RIGHT_INDEX_Y',
 'RIGHT_INDEX_Z',
 'LEFT_THUMB_X',
 'LEFT_THUMB_Y',
 'LEFT_THUMB_Z',
 'RIGHT_THUMB_X',
 'RIGHT_THUMB_Y',
 'RIGHT_THUMB_Z',
 'LEFT_HIP_X',
 'LEFT_HIP_Y',
 'LEFT_HIP_Z',
 'RIGHT_HIP_X',
 'RIGHT_HIP_Y',
 'RIGHT_HIP_Z',
 'LEFT_KNEE_X',
 'LEFT_KNEE_Y',
 'LEFT_KNEE_Z',
 'RIGHT_KNEE_X',
 'RIGHT_KNEE_Y',
 'RIGHT_KNEE_Z',
 'LEFT_ANKLE_X',
 'LEFT_ANKLE_Y',
 'LEFT_ANKLE_Z',
 'RIGHT_ANKLE_X',
 'RIGHT_ANKLE_Y',
 'RIGHT_ANKLE_Z',
 'LEFT_HEEL_X',
 'LEFT_HEEL_Y',
 'LEFT_HEEL_Z',
 'RIGHT_HEEL_X',
 'RIGHT_HEEL_Y',
 'RIGHT_HEEL_Z',
 'LEFT_FOOT_INDEX_X',
 'LEFT_FOOT_INDEX_Y',
 'LEFT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_X',
 'RIGHT_FOOT_INDEX_Y',
 'RIGHT_FOOT_INDEX_Z']


colX = []
colY = []
colZ = []

for col in X_cols :
     if 'X' in col:
          colX.append(col)
     elif 'Y' in col:
          colY.append(col)
     elif 'Z' in col:
          colZ.append(col)
          
target = 'EXERCICE'
encode = LabelEncoder()

NameError: name 'LabelEncoder' is not defined

In [ ]:
X_train = df_train[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_train = encode.fit_transform(df_train[target])

In [ ]:
X_test = df_test[X_cols].apply(lambda x : x - np.mean(x),axis=1)
y_test = encode.transform(df_test[target])

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64,activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32,activation = 'relu'))
model.add(tf.keras.layers.Dense(4,activation='softmax'))

In [ ]:
model.compile(loss = 'SparseCategoricalCrossentropy')

In [ ]:
model.fit(X_train,y_train,epochs = 200,validation_data= (X_test,y_test))

Epoch 1/200
92/92 [==============================] - 1s 3ms/step - loss: 1.1144 - val_loss: 0.8177
Epoch 2/200
92/92 [==============================] - 0s 2ms/step - loss: 0.8662 - val_loss: 0.6948
Epoch 3/200
92/92 [==============================] - 0s 2ms/step - loss: 0.7891 - val_loss: 0.6270
Epoch 4/200
92/92 [==============================] - 0s 2ms/step - loss: 0.7173 - val_loss: 0.5685
Epoch 5/200
92/92 [==============================] - 0s 2ms/step - loss: 0.6864 - val_loss: 0.5476
Epoch 6/200
92/92 [==============================] - 0s 2ms/step - loss: 0.6584 - val_loss: 0.5161
Epoch 7/200
92/92 [==============================] - 0s 1ms/step - loss: 0.6276 - val_loss: 0.4793
Epoch 8/200
92/92 [==============================] - 0s 1ms/step - loss: 0.6370 - val_loss: 0.4703
Epoch 9/200
92/92 [==============================] - 0s 1ms/step - loss: 0.6025 - val_loss: 0.4617
Epoch 10/200
92/92 [==============================] - 0s 2ms/step - loss: 0.6002 - val_loss: 0.4316
Epoch 11/

In [ ]:
proba_model = model.predict(X_test)
predictions = np.argmax(proba_model,axis=1)
metrics = tf.keras.metrics.Accuracy()

In [ ]:
print('Accuracy : {}'.format(metrics(y_test,predictions)))

Accuracy : 0.9076336026191711


In [ ]:
with open('./Model/encoder3.pkl','wb') as file:
    pickle.dump(encode,file)

In [ ]:
model.save('./Model/model3')

INFO:tensorflow:Assets written to: ./Model/model2\assets
